# Example Live Updates
### Could pass Excel file through a callback to update graphs and charts as new data is added
#### The data should be passed through the callback first
###### Could also use Bloomberg API instead of Yahoo Finance

In [1]:
#### You might need to pip upgrade poltly and dash
import yfinance as yf
import pandas_datareader
from plotly.subplots import make_subplots
from dash.dependencies import Input, Output
from yahoo_fin import stock_info as si
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
from plotly.express import *
import plotly.graph_objects as go
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from pytickersymbols import PyTickerSymbols
from yahooquery import Ticker
import pandas_ta as ta
import dash
from dash import dcc, html, dash_table
import plotly
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import requests
import json
yf.pdr_override()

#### Data - Used for the dropdown (Russell 3000 ETF Holdings)
###### There will be tickers that are not equity (e.g., cash and futures)
###### If you don't want to use a dropdown, you can use dcc.Input and type the ticker - I will add that option in the layout and you can test it out (just # the dropdown)

In [2]:
url = 'https://www.ishares.com/us/products/239714/ishares-russell-3000-etf/us/products/239714/ishares-russell-3000-etf/1467271812596.ajax?tab=all&fileType=json'
r = requests.get(url)
r.encoding='utf-8-sig'
jsonData = json.loads(r.text)

rows = []
for each in jsonData['aaData']:
    row = {'Ticker':each[0],
#      'Name':each[1],
#      'Sector':each[2],
#      'Asset Class':each[3],
#      'Market Value':each[4]['display'],
#      'Market Value Raw':each[4]['raw'],
#      'Weight (%)':each[5]['display'],
#      'Weight (%) Raw':each[5]['raw'],
#      'Notaional Value':each[6]['display'],
#      'Notaional Value Raw':each[6]['raw'],
#      'Nominal':each[7]['display'],
#      'Nominal Raw':each[7]['raw'],
#      'ISIN':each[8],
#      #'Price':each[9]['display'],
#      #'Price Raw':each[9]['raw'],
#      'Location':each[10],
#      'Exchange':each[11],
#      'Market Currency':each[12]
          }
     
    rows.append(row)
     
df = pd.DataFrame(rows)

## Tickers:
Tickers = df['Ticker']

ticker_options = []
for Ticker in Tickers.unique():
    ticker_options.append({'label': str(Ticker), 'value': Ticker})
    
Tickers

0         AAPL
1         MSFT
2         AMZN
3         TSLA
4        GOOGL
         ...  
2691      GTXI
2692    P5N994
2693       TFM
2694      ESZ2
2695     RTYZ2
Name: Ticker, Length: 2696, dtype: object

##### Chart Layout

In [3]:
layout = load_figure_template('DARKLY')

### App
#### The most important detail here is the dcc.Interval

In [4]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.layout = html.Div([
    html.Br(),
    html.H1('Example of Live Update', style={'text-align': 'center', 'display':'inline-block', 'width':'100%'}),
    html.Hr(),
    html.Br(),
    dcc.Interval(
                id='interval',
                disabled=False,
                interval=1.5*10000, #Updated every 15 seconds (this is in milliseconds)
                n_intervals=0,
                max_intervals=-1, #-1 is unilimited times of updates - if = 4, it's updated only 4 times
    ),
    html.Div([dcc.Dropdown(id='Select_Ticker',
                               options=ticker_options,
                               value='TSLA',
                               placeholder='Ticker',
                               style={'color': 'black'}),
                  ], style={'width': '50%', 'text-align': 'center', 'margin-left': '7px'}),
#     html.Div([dcc.Input(id='Select_Ticker', placeholder='Select Ticker (e.g., SPY)')],
#             style={'width': '50%', 'text-align': 'center', 'margin-left': '7px'}), 
    ### NOTE THAT FX WON'T HAVE CALLS/PUTS TAB
    html.Br(),
    html.Div([html.Img(id="logo")], style={'width': '10%', 'margin-left': '10px'}),
    html.Div([html.P(id="ticker")], style={'width': '90%',
                                           'margin-left': '7px', 
                                           'font-family': 'sans-serif', 'fontSize': 14}),
    html.Br(),
    html.Div([(dcc.Graph(id='chart',
                         style={'width': "100%", 'height': '120%', "display": "inline-block", 'text-align': 'center'}))],
             style = {'width':'50%','text-align': 'center', "display": "inline-block"}),
    html.Div([(dcc.Graph(id='chart_1',
                         style={'width': "100%", 'height': '120%', "display": "inline-block", 'text-align': 'center'}))],
             style = {'width':'50%','text-align': 'center', "display": "inline-block"}),
    html.Br(),
    html.Div([
        dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', children=[
            dcc.Tab(label='Calls', value='tab-1-example-graph'),
            dcc.Tab(label='Puts', value='tab-2-example-graph'),
        ], colors={"border":"gold", "primary":"#55595c", "background":"black"}),
        html.Div(id='tabs-content-example-graph')
    ]),
    html.Br(),
    html.Hr()
])

In [5]:
@app.callback(Output("logo", "src"),
              Output("ticker", "children"),
              [Input("Select_Ticker", "value")])
def update_data(cur):
    if cur == None:
        raise PreventUpdate

    ticker = yf.Ticker(cur)
    inf = ticker.info

    df = pd.DataFrame.from_dict(inf, orient="index").T
    df = df[["logo_url", "shortName"]]

    return df["logo_url"].values[0], df["shortName"].values[0]

#### Callback - Chart

In [6]:
@app.callback(
    Output("chart", "figure"),
    Output('chart_1', 'figure'),
    [Input("interval", "n_intervals")],
    [Input("Select_Ticker", "value")])
def update_graph(num, cur):
    if num == 0:
        raise PreventUpdate
    else:
        df = yf.download(tickers=cur, period='1d', interval='1m')
        df = pd.DataFrame(df)
        df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)
        df.ta.rsi(append=True)
        df.ta.bbands(close='Close', length=20, std=2, append=True)
        fig = go.Figure()
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                            vertical_spacing=0.03,
                            row_heights=[5, 2])
        fig.add_trace(go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='Price'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index,
                                 y=df['BBL_20_2.0'],
                                 opacity=0.7,
                                 line=dict(color='yellow', width=2),
                                 name='BBL'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index,
                                 y=df['BBM_20_2.0'],
                                 opacity=0.7,
                                 line=dict(color='purple', width=2),
                                 name='BBM'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index,
                                 y=df['BBU_20_2.0'],
                                 opacity=0.7,
                                 line=dict(color='blue', width=2),
                                 name='BBU'), row=1, col=1)
        colors = ['green' if row['Open'] - row['Close'] >= 0
                  else 'red' for index, row in df.iterrows()]
        fig.add_trace(go.Bar(x=df.index,
                             y=df['Volume'],
                             marker_color=colors,
                             showlegend=False),
                      row=2, col=1)
        fig.update_layout(
            title=str(cur) + ' Live Share Price',
            yaxis_title='Stock Price (USD per Shares)')
        fig.update_yaxes(title_text="Price", row=1, col=1)
        fig.update_yaxes(title_text="Volume", row=2, col=1)
        fig.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=15,
                             label="15min",
                             step="minute",
                             stepmode="backward"),
                        dict(count=45,
                             label="45min",
                             step="minute",
                             stepmode="backward"),
                        dict(count=1,
                             label="HTD",
                             step="hour",
                             stepmode="todate"),
                        dict(count=3,
                             label="3h",
                             step="hour",
                             stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=False,
                ),
                type="date"
            )
        )
        fig.update_layout(yaxis_tickformat='$')
        fig.update_layout(template='plotly_dark',
                          xaxis_rangeselector_font_color='black',
                          xaxis_rangeselector_activecolor='red',
                          xaxis_rangeselector_bgcolor='green',
                          )
        # prevents layout to change every second (e.g., if you select 15min view)
        fig.update_layout(uirevision='constant')
        fig.update_layout(xaxis=dict(showgrid=False))

        ###########################################################

        fig1 = go.Figure()
        fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True,
                             vertical_spacing=0.03,
                             row_heights=[5, 2])
        fig1.append_trace(go.Scatter(
            x=df.index,
            y=df['MACD_12_26_9'],
            opacity=0.7,
            line=dict(color='blue', width=2),
            name='MACD'), row=1, col=1)
        fig1.append_trace(go.Scatter(x=df.index,
                                     y=df['MACDs_12_26_9'],
                                     opacity=0.7,
                                     line=dict(color='orange', width=2),
                                     name='MACDs'), row=1, col=1)
        colorsM = ['green' if val >= 0
                   else 'red' for val in df['MACDh_12_26_9']]

        fig1.add_trace(go.Bar(x=df.index,
                              y=df['MACDh_12_26_9'],
                              marker_color=colorsM,
                              showlegend=False
                              ), row=1, col=1)
        colorsR = ['green' if val >= 50
                   else 'red' for val in df['RSI_14']]
        fig1.add_trace(go.Bar(x=df.index,
                              y=df['RSI_14'],
                              marker_color=colorsR,
                              showlegend=False),
                       row=2, col=1)
        fig1.add_hline(y=50, col=1, row=2, line_color="#666", line_width=2)

        fig1.update_layout(
            title=str(cur) + ' Technical Indicators')
        fig1.update_yaxes(title_text="MACD", row=1, col=1)
        fig1.update_yaxes(title_text="RSI", row=2, col=1)
        fig1.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=15,
                             label="15min",
                             step="minute",
                             stepmode="backward"),
                        dict(count=45,
                             label="45min",
                             step="minute",
                             stepmode="backward"),
                        dict(count=1,
                             label="HTD",
                             step="hour",
                             stepmode="todate"),
                        dict(count=3,
                             label="3h",
                             step="hour",
                             stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=False,
                ),
                type="date"
            )
        )
        fig1.update_layout(template='plotly_dark',
                           xaxis_rangeselector_font_color='black',
                           xaxis_rangeselector_activecolor='red',
                           xaxis_rangeselector_bgcolor='green',
                           )
        # prevents layout to change every second (e.g., if you select 15min view)
        fig1.update_layout(uirevision='constant')
        fig1.update_layout(xaxis=dict(showgrid=False))

    return fig, fig1

#### Callback - Table

In [7]:
@app.callback(Output('tabs-content-example-graph', 'children'),
              [Input("interval", "n_intervals")],
              [Input('tabs-example-graph', 'value')],
              [Input("Select_Ticker", "value")])
def render_content(num, tab, cur):
    if num == 0:
        raise PreventUpdate
    else:
        if tab == 'tab-1-example-graph':
            Ticker = yf.Ticker(cur)
            options = Ticker.option_chain()
            calls = pd.DataFrame(options.calls)
            calls = calls.nlargest(10, 'openInterest')
            columns = [{"name": i, 'type': 'numeric', 'format': {'specifier': '.2f'}, "id": i} for i
                       in calls.columns]
            calls = calls.to_dict('records')
            return html.Div([dash_table.DataTable(data=calls, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])

        elif tab == 'tab-2-example-graph':
            Ticker = yf.Ticker(cur)
            options = Ticker.option_chain()
            puts = pd.DataFrame(options.puts)
            puts = puts.nlargest(10, 'openInterest')
            columns = [{"name": i, 'type': 'numeric', 'format': {'specifier': '.2f'}, "id": i} for i
                       in puts.columns]
            puts = puts.to_dict('records')
            return html.Div([dash_table.DataTable(data=puts, columns=columns,
                                       style_cell={'textAlign': 'center', 'font-family': 'sans-serif', 'fontSize': 12},
                                       style_header={
                                           'backgroundColor': '#939996',
                                           'fontWeight': 'bold'},
                                       style_data={
                                           'color': 'white',
                                           'backgroundColor': 'black',
                                           'whiteSpace': 'normal',
                                           'height': '5px',
                                           'overflowY': 'auto',
                                       })])

#### Start App

In [8]:
if __name__ == '__main__':
    app.run(debug=False, use_reloader=False, port=4321) #make sure port is available 

Dash is running on http://127.0.0.1:4321/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
